In [20]:
####################################################################
##  
##  ETL PROJECT - TEAM 5
##          - Fatma Butun (Chicago Data)
##          - George Alonzo (Los Angeles Data)
##
##
####################################################################

In [21]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

# OVERVIEW

This project will perform the ETL process on 2 CSV files relating to **2020Q1** bike share data from the **City of Chicago** and the **City of Los Angeles**.  *(NOTE: We are only working with 2020Q1 data as it is the most recent provided by the City of Chicago.)*  

We will begin by loading each CSV file into separate, initial dataframes.  From there, we will begin the transformation process by focusing on only the required columns and add unique prefixes & static city code, where needed, to differentiate between the two data sets once they are loaded into the same table.  Date/timestamps will be split into two separate columns and the duration will be calculated for where it is not already provided.  Columns will be renamed and reordered to remain consistent as we prep to insert into the SQL tables.

We will store station information separately into its own SQL table to avoid redundant data.  Since the station IDs are defined as integers in both files, we will add a city prefix to the station ID to ensure they are identifyable to the city in which they belong.

### PREREQUISITES:  
Creation of the SQL database and tables must be completed prior to running this notebook.  Please see ``readme`` for more information.

# EXTRACT

##### This section reads each of the CSV files and stores into separate dataframes.

## *LOS ANGELES*

In [22]:
# Read-in Los Angeles ride share data into an initial dataframe
la_csv = 'data/metro-bike-share-trips-2020-q1.csv'
la_csv_df = pd.read_csv(la_csv)
#la_csv_df.head()

## *CHICAGO*

In [23]:
# reda the csv into a dataframe
chi_bikeshare = pd.read_csv("data/Divvy_Trips_2020_Q1.csv")
# check dataframe
#chi_bikeshare.head()

# TRANSFORM

##### This section processes the initial dataframes created from the Extract section and prepares for the Load section.

## *LOS ANGELES*

##### *Bike Share Data*

In [24]:
# Prefix the station IDs with a city code to make the station IDs unique.
# Add static city name to differentiate records within the same table in the database
la_csv_df['city_name'] = 'Los Angeles'
la_csv_df['start_station'] = 'LA-'+la_csv_df['start_station'].astype(str)
la_csv_df['end_station'] = 'LA-'+la_csv_df['end_station'].astype(str)
#la_csv_df.head()

In [25]:
# Slim-down the dataframe to just the required columns for the rideshare table
la_slim_df = la_csv_df[['trip_id','start_time','end_time','start_station','end_station','duration','passholder_type','city_name']].copy()
#la_slim_df.head()

In [26]:
# Rename columns to be consistent amongst the two data sources
la_renamed_df = la_slim_df.rename(columns={'trip_id': 'ride_id', 'start_time':'start_dt','end_time':'end_dt','start_station': 'start_station_id','end_station':'end_station_id','passholder_type':'membership_type'})
#la_renamed_df.head()

In [27]:
# Split start start & end datetime columns into separate date & time columns
la_renamed_df['start_date'] = pd.to_datetime(la_renamed_df['start_dt']).dt.date
la_renamed_df['start_time'] = pd.to_datetime(la_renamed_df['start_dt']).dt.time
la_renamed_df['end_date'] = pd.to_datetime(la_renamed_df['end_dt']).dt.date
la_renamed_df['end_time'] = pd.to_datetime(la_renamed_df['end_dt']).dt.time
#la_renamed_df.head()

In [28]:
# Drop the original start & end datetimestamp columns
la_renamed_df = la_renamed_df.drop(['start_dt'],axis=1)
la_renamed_df = la_renamed_df.drop(['end_dt'],axis=1)
#la_renamed_df.head()

In [29]:
# Reorder fields to be consistent amongst the two dataframes
la_bikeshare_df = la_renamed_df[['ride_id','start_station_id','start_date','start_time',
                                 'end_station_id','end_date','end_time','duration','membership_type',
                                'city_name']]

#la_bikeshare_df.head()

##### *Station Data*

In [30]:
# Slim-down the dataframe to just the required columns for the station table
la_start_stations= la_csv_df[['start_station','start_lat','start_lon']]
la_start_stations = la_start_stations.rename(columns={'start_station': 'station_id', 'start_lat': 'latitude','start_lon': 'longitude'})


#la_start_stations.head()

In [31]:
# Slim-down the dataframe to just the required columns for the station table
la_end_stations= la_csv_df[['end_station','end_lat','end_lon']]
la_end_stations = la_end_stations.rename(columns={'end_station': 'station_id', 'end_lat': 'latitude','end_lon': 'longitude'})
#la_end_stations.head()

In [32]:
# Concatenate both the start & end dataframes into a single df
la_station_df = pd.concat([la_start_stations,la_end_stations])
# Drop any duplicates
la_station_df = la_station_df.drop_duplicates()
# WE ARE NOT DROPPING ANY NaN ROWS AS THIS WILL CAUSE ERRORS DURING THE
#    INSERT PROCESS AS THE STATION IS DESIGNATED AS A FOREIGN KEY 
#la_station_df = la_station_df.dropna()
#la_station_df

## *CHICAGO*

In [33]:
# rename columns to have them in unisome with the other dataframe

chi_bikeshare = chi_bikeshare.rename(columns={"started_at":"start_time", "ended_at":"end_time", "member_casual":"membership_type"})

In [34]:
# append CHI- to the station IDs to prevent any duplicate IDs between two different dataframes

chi_bikeshare['start_station_id'] = 'CHI-' + chi_bikeshare['start_station_id'].astype(str) 
chi_bikeshare['end_station_id'] = 'CHI-' + chi_bikeshare['end_station_id'].astype(str) 
#chi_bikeshare.head()

Make a table that contains all the station IDs and their latitude and longitude information

In [35]:
# extract relavant columns for the station id df
chi_station_id_s= chi_bikeshare[["start_station_id", "start_lat", "start_lng"]]
chi_station_id_e= chi_bikeshare[["end_station_id", "end_lat", "end_lng"]]

In [36]:
# rename column names so that both station df will have the same columns
chi_station_id_s =chi_station_id_s.rename(columns={"start_station_id":"station_id", "start_lat":"latitude","start_lng": "longitude"})

In [37]:
#rename column names so that both station df will have the same columns
chi_station_id_e= chi_station_id_e.rename(columns={"end_station_id":"station_id", "end_lat":"latitude","end_lng": "longitude"})

In [38]:
# combine two df in one and drop the duplicate station_ids
chi_station_id = chi_station_id_e.append(chi_station_id_s)
chi_station_id = chi_station_id.drop_duplicates(subset=['station_id'])
#chi_station_id["station_id"].nunique()
#chi_station_id.head()

Make a table that contains information about chicago bike share. The dataframe will include information about ride_id, start_station_id, start_date, start_time, end_station_id, end_date, end_time, duration , membership_type and city_name.

In [39]:
# continue with the main bikeshare dataframe.  keep relavant columns

chi_bikeshare =chi_bikeshare[["ride_id", "start_time", "end_time", "start_station_id", "end_station_id","membership_type" ]]
#chi_bikeshare.head()

In [40]:
# make duplicate columns of start_time and end_time to use later

chi_bikeshare["start_tm"] = chi_bikeshare["start_time"]
chi_bikeshare["end_tm"] = chi_bikeshare["end_time"]

In [41]:
# Change the start_time and end_time columns into datetime64 in order to calculate the duration
chi_bikeshare["end_time"] = pd.to_datetime(chi_bikeshare["end_time"])
chi_bikeshare["start_time"] = pd.to_datetime(chi_bikeshare["start_time"])

#chi_bikeshare.head()

In [42]:
# Calculate the duration of each bike ride
chi_bikeshare["duration"] = (chi_bikeshare["end_time"] - chi_bikeshare["start_time"]).astype('timedelta64[m]')
#chi_bikeshare.head()

In [43]:
# add the city name column and fill with "Chicago"
chi_bikeshare["city_name"] = "Chicago"
#chi_bikeshare.head()

In [44]:
a = chi_bikeshare
#a.head()

In [45]:
# drop the start time and end time columns that are in datetype.
a= a.drop(columns=["start_time", "end_time"])
#a.head()

In [46]:
# seperate the start and end tm columns into 2 columns each

a[['start_date', 'start_time']] = a['start_tm'].str.split(' ', 1, expand=True)
a[['end_date', 'end_time']] = a['end_tm'].str.split(' ', 1, expand=True)
#a.head()

In [47]:
# keep only required columns and rearrange the order

chicago_bikeshare_df= a[["ride_id", "start_station_id", "start_date", "start_time", "end_station_id", "end_date", "end_time", "duration" , "membership_type", "city_name"]]
# check final dataframe
#chicago_bikeshare_df.head()

# LOAD

##### This section creates the connection to the SQL database and inserts data

In [51]:
# create a connection string to postgres
connection_string = "postgres:bootcamp@localhost:5432/bikeshare"
engine = create_engine(f'postgresql://{connection_string}')

In [52]:
# check the tables in thebikeshare database
engine.table_names()

<ipython-input-52-6abbb41c1372>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['station', 'bike_trip']

## *LOS ANGELES*

In [53]:
la_station_df.to_sql(name='station', con=engine, if_exists='append', index=False)

In [54]:
la_bikeshare_df.to_sql(name='bike_trip', con=engine, if_exists='append', index=False)

## *CHICAGO*

In [55]:
chi_station_id.to_sql(name='station', con=engine, if_exists='append', index=False)

In [56]:
chicago_bikeshare_df.to_sql(name='bike_trip', con=engine, if_exists='append', index=False)

# BASIC DATABASE CHECKING & VALIDATIONS

#####  This section performs a quick check to validate that data was properly inserted into the table.

In [59]:
pd.read_sql_query('select * from bike_trip', con=engine).head()

,ride_id,start_station_id,start_date,start_time,end_station_id,end_date,end_time,duration,membership_type,city_name
0,134867493,LA-3063,2020-01-01,00:16:00,LA-4491,2020-01-01,00:41:00,25.0,Monthly Pass,Los Angeles
1,134867799,LA-4285,2020-01-01,00:24:00,LA-4354,2020-01-01,00:59:00,35.0,One Day Pass,Los Angeles
2,134868104,LA-4344,2020-01-01,00:31:00,LA-4322,2020-01-01,01:08:00,37.0,Walk-up,Los Angeles
3,134868103,LA-4344,2020-01-01,00:32:00,LA-4322,2020-01-01,01:08:00,36.0,Walk-up,Los Angeles
4,134868102,LA-4344,2020-01-01,00:33:00,LA-4322,2020-01-01,01:08:00,35.0,Walk-up,Los Angeles


In [60]:
pd.read_sql_query('select city_name, count(*) from bike_trip group by city_name', con=engine).head()

,city_name,count
0,Chicago,426887
1,Los Angeles,76391


In [62]:
pd.read_sql_query('select * from station order by latitude', con=engine).head()

,station_id,latitude,longitude
0,LA-4403,33.943359,-118.248238
1,LA-4504,33.958790,-118.448280
2,LA-4469,33.970242,-118.426071
3,LA-4496,33.972980,-118.423943
4,LA-4470,33.976189,-118.418419
